In [ ]:
# PAR_load_F1.4_python2

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors

import numpy as np
from netCDF4 import Dataset
from scipy.interpolate import griddata
from mpl_toolkits.basemap import Basemap

#from PiecewiseNorm import PiecewiseNorm
#import os.path
#import scipy.io as spio
#import pickle

modulepath   = '/home/ollie/mozeisin/pyfesom/'

import sys
sys.path.append(modulepath)

import pyfesom as pf1

# load FESOM 1.4 data

In [ ]:
resultpath = '/work/ollie/ncara/fesom/fesom-1.4-recom/v3/reccap_A_varCO2_varclim_1958_2019/'

#meshpath     = '/work/ollie/jhauck/input/meshes/CORE2/'
meshpath     = '/work/ollie/jhauck/input/meshes/core_new_288/'
#meshpath     = '/work/ollie/jhauck/input/meshes/core_new_384/'

# period of analysis 
first_year = 1960
last_year  = 1970
    
# number of upper ocean layers for surface mean
n_levels = 1

years = [first_year,last_year]
years_list      = np.arange(first_year,last_year+1,1)
months     = np.linspace(0,11,12).astype(int)

# dump data to this folder
dump = False
dumppath = '/work/ollie/mozeisin/evaluation/temp_data/FESOM1.4_'+str(first_year)+'_'+str(last_year)+'_'

In [ ]:
# load mesh --------------------------------------------------------------------------------------------
mesh = pf1.load_mesh(meshpath,abg=[0,0,0],get3d=True,usepickle=False)

In [ ]:
print(mesh.n2d)
#print(len(mesh.x2))

In [ ]:
# overview nc variables
ncfilePhyChl = resultpath+var_PhyChl+'_'+runid+'_'+str(years_list[ind])+'0101.nc'

In [ ]:
# define FESOM output variables to use --------------------------------------------------------------------------------------------
runid='fesom'

var_PAR = '' # []

# define storage arrays --------------------------------------------------------------------------------------------
PAR = np.zeros(len(mesh.x2))


#print(np.shape(PAR))

# load data --------------------------------------------------------------------------------------------
for ind in range(0,len(years_list)):
    print(years_list[ind])
    
    # PAR
    ncfile = resultpath+var_PAR+'_'+runid+'_'+str(years_list[ind])+'0101.nc'
    f      = Dataset(ncfile, 'r')
    PARaux    = f.variables[var_PAR][:].mean(axis=0) # annual mean
    #print(np.shape(PARaux[0:mesh.n2d]))
    PAR  = PAR + PARaux[0,0:mesh.n2d] # select only surface layer
    #print(np.shape(PAR))

   
    
# data means --------------------------------------------------------------------------------------------    
PAR_mean = PAR/len(years_list)


#print(np.shape(PAR_mean))


# Control: Plotting on original grid (basemap)

In [ ]:
elem2=mesh.elem[mesh.no_cyclic_elem,:]
d=PhyChl2D_mean[elem2].mean(axis=1)
k = [i for (i, val) in enumerate(d) if not np.isnan(val)]
elem2=elem2[k,:]
print 'ftriplot, number of dummy points:', len(d)-len(k) 

fig = plt.figure(num=18, figsize=(8, 8), dpi=300, facecolor='w', edgecolor='k')
   
map = Basemap(projection='robin',lon_0=0)
x, y = map(mesh.x2, mesh.y2)
map.drawmapboundary(fill_color='0.9')
map.drawcoastlines()
mlabels=[False,False,False,False]
plabels=[True,True,True,True]
map.drawparallels(np.arange(-90,90,30),labels=plabels) #[1,0,0,0]
map.drawmeridians(np.arange(map.lonmin,map.lonmax+30,60),labels=mlabels) #[0,0,0,1]

contoursDChl=np.array([0,0.02,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.18,0.2,0.22,0.24,0.26,0.28,0.3,0.32,0.34,0.36,0.38,0.4,0.42,0.44,0.46,0.48,0.5,0.52,0.54,0.56,0.58,0.6])
eps=(contoursDChl.max()-contoursDChl.min())/50.
#data2[data2<=contours.min()]=contours.min()+eps
#data2[data2>=contours.max()]=contours.max()-eps

imDChl=plt.tricontourf(x, y, elem2, PAR_mean, 
                       #levels=contoursDChl, 
                       cmap='viridis', 
                       #norm=PiecewiseNorm(contoursDChl), 
                       extend='max')

labelPAR = 'PAR ('+str(years[0])+' to '+str(years[len(years)-1])+')' # in case of multiple years, replace ('+str(years[0])+') by ('+str(years[0])+' to '+str(years[len(years)-1])+')
plt.title(labelPAR)
cbarDChl=map.colorbar(imDChl,"bottom", size="5%", pad="2%")
cbarDChl.set_label(r'[]') 

# Interpolation to regular grid

In [ ]:
lat         = np.arange(-89.5,90.,1.)
lon         = np.arange(-179.5,180.,1.)
latdic, londic = np.meshgrid(lat, lon)

lon_fesom = mesh.x2
lat_fesom = mesh.y2

print(np.shape(lon_fesom), np.shape(lat_fesom))

# PAR check

In [ ]:
PAR_mean_interp = griddata( (lon_fesom, lat_fesom), PAR_mean.ravel(),
                              (londic, latdic), method='nearest')

PAR_mean_interp = np.ma.filled(PAR_mean_interp, np.nan)

print('\nFESOM PAR interpolation check [?]:\noriginal min {0:5.4f} max {1:5.4f} mean {2:5.4f} \ninterpol min {3:5.4f} max {4:5.4f} mean {5:5.4f}'.format(
                np.nanmin(PAR_mean),np.nanmax(PAR_mean),np.nanmean(PAR_mean),
                np.nanmin(PAR_mean_interp),np.nanmax(PAR_mean_interp),np.nanmean(PAR_mean_interp)))
    

In [ ]:
### Interpolation check
figDChl = plt.figure(num=18, figsize=(8, 8), dpi=300, facecolor='w', edgecolor='k')
   
map = Basemap(projection='robin',lon_0=0)
x, y = map(londic, latdic)
map.drawmapboundary(fill_color='0.9')
map.drawcoastlines()
mlabels=[False,False,False,False]
plabels=[True,True,True,True]
map.drawparallels(np.arange(-90,90,30),labels=plabels) #[1,0,0,0]
map.drawmeridians(np.arange(map.lonmin,map.lonmax+30,60),labels=mlabels) #[0,0,0,1]

#contoursDChl=np.array([0,0.02,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.18,0.2,0.22,0.24,0.26,0.28,0.3,0.32,0.34,0.36,0.38,0.4,0.42,0.44,0.46,0.48,0.5,0.52,0.54,0.56,0.58,0.6])
eps=(contoursDChl.max()-contoursDChl.min())/50.
#data2[data2<=contours.min()]=contours.min()+eps
#data2[data2>=contours.max()]=contours.max()-eps

imDChl=plt.pcolormesh(x, y, PAR_mean, 
                      #norm=colors.BoundaryNorm(boundaries=contoursDChl, ncolors=256), 
                      cmap='viridis', 
                       #norm=PiecewiseNorm(contoursDChl), 
                       )

labelDChl = 'PAR ('+str(years[0])+' to '+str(years[len(years)-1])+')' # in case of multiple years, replace ('+str(years[0])+') by ('+str(years[0])+' to '+str(years[len(years)-1])+')
plt.title(labelDChl)
cbarDChl=map.colorbar(imDChl,"bottom", size="5%", pad="2%")
cbarDChl.set_label(r'[]')

# export data to temp_data

In [ ]:
if dump:
    np.save(dumppath + 'PAR.npy',PAR_mean_interp, allow_pickle=False)
    